# Load in the SVV Connection

In [1]:
import psycopg2

def get_svv_connection():
    con=psycopg2.connect(dbname= 'svv', host='svv-rs-prod-bi.cjddijbnvfpr.eu-west-1.redshift.amazonaws.com', \
                         port= 5439, user= 'ryanw', password= 'hT6Y3TeZZUFdvLj')
    
    return con

# Import the Necessary Packages

In [2]:
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from sklearn.decomposition import PCA, FastICA, FactorAnalysis, NMF
from sklearn.preprocessing import StandardScaler
from scipy import stats

# Connect to SVV and Query the Relevant Tables

In [3]:
query_data_1 = "Select * from sandbox.genre_consumption_proportions_hubplus_segmentation_AB"
query_data_2 = "Select * from sandbox.subgenre_consumption_proportions_hubplus_segmentation_AB"
query_data_3 = "Select * from sandbox.viewers_hubplus_segmentation_AB"
query_data_4 = "Select * from sandbox.programme_proportions_hubplus_segmentation_AB"
query_data_5 = "Select * from sandbox.viewer_demographics_hubplus_segmentation_AB"

df_genre = pd.DataFrame()

try: 
    con = get_svv_connection()

    cur = con.cursor()
    cur.execute(query_data_1)

    colnames = [desc[0] for desc in cur.description]
    df_genre = pd.DataFrame(cur.fetchall(), columns=colnames)
    
    cur.execute(query_data_2)
    
    colnames = [desc[0] for desc in cur.description]
    df_subgenre = pd.DataFrame(cur.fetchall(), columns=colnames)
    
    cur.execute(query_data_3)
    
    colnames = [desc[0] for desc in cur.description]
    df_viewers = pd.DataFrame(cur.fetchall(), columns=colnames)
    
    cur.execute(query_data_4)
    
    colnames = [desc[0] for desc in cur.description]
    df_programme = pd.DataFrame(cur.fetchall(), columns=colnames)
    
    cur.execute(query_data_5)
    
    colnames = [desc[0] for desc in cur.description]
    df_demographics = pd.DataFrame(cur.fetchall(), columns=colnames)
    
    
except Exception as inst:
    print(inst)
finally:
    cur.close()
    con.close()

# Make Sure that the Queries Worked and Returned the Right Outputs

In [4]:
df_genre.head()

,viewer_id,genre_id,genre_proportion_consumption
0,00036abb-599a-4a5c-b1c0-d084b5f01b5f,DRAMA,1.0000
1,00036abb-599a-4a5c-b1c0-d084b5f01b5f,None,None
2,00042180-268e-4f50-943f-16aa79f15e67,ENT,0.9892
3,00042180-268e-4f50-943f-16aa79f15e67,COMEDY,0.0055
4,00042180-268e-4f50-943f-16aa79f15e67,FACTUAL,0.0052


In [5]:
df_subgenre.head()

,viewer_id,sub_genre,subgenre_proportion_consumption
0,00010645-0c19-4a21-8b5f-112ca12b2f22,LONG-RUNNING DRAMA,0.1951
1,00010645-0c19-4a21-8b5f-112ca12b2f22,FACTUAL ENTERTAINMENT,0.3538
2,00010645-0c19-4a21-8b5f-112ca12b2f22,CELEBRITIES,0.0121
3,00010645-0c19-4a21-8b5f-112ca12b2f22,QUIZ AND GAME SHOWS,0.0121
4,00010645-0c19-4a21-8b5f-112ca12b2f22,FOOD AND DRINK,0.0040


In [6]:
df_viewers.head()

,viewer_id,tenure_months
0,f94aac52-0e87-410c-ad36-330ed144de9f,1
1,01e385da-1954-46da-9035-f74941b7e0d5,3
2,03c4091c-24b2-4191-9356-e518b5413bb4,6
3,0cfa767b-fcf1-41a9-ac15-828fb6874b16,1
4,11ce95ab-4d8a-4a62-9fef-54caffe8ad50,2


In [7]:
df_programme.head()

,viewer_id,programme_title,programme_proportion_consumption
0,00010645-0c19-4a21-8b5f-112ca12b2f22,The Only Way is Essex,0.0701
1,00010645-0c19-4a21-8b5f-112ca12b2f22,Love Island: Aftersun,0.0069
2,00010645-0c19-4a21-8b5f-112ca12b2f22,Absolutely Ascot,0.0263
3,00010645-0c19-4a21-8b5f-112ca12b2f22,Love Island,0.1528
4,00010645-0c19-4a21-8b5f-112ca12b2f22,Love Island: Australia,0.2810


In [8]:
df_demographics.head()

,viewer_id,age,gender,abc1_flag,h_lifestage,pc_mosaic_uk_6_type,pc_mosaic_uk_6_group
0,008e9dde-3724-4b38-afe8-ebe77a10936e,55.0,M,1,09,25,F
1,0199f9be-fc50-42d8-af74-8684a01ce8a3,34.0,M,1,03,33,H
2,01fdb9cd-355d-4cf1-a811-f78d53d0038e,92.0,M,0,None,None,None
3,026346f3-9f29-48c6-b71e-442f1b4fb971,52.0,F,1,None,None,None
4,027b4507-7a5a-42c2-b8ee-7a0893999298,38.0,M,1,09,02,A


# Clean up the Multiple Dataframes

In [9]:
#drop NAs from the genre, subgenre and programme tables

df_genre = df_genre.dropna(subset=['genre_id', 'genre_proportion_consumption'], how='all')
df_subgenre = df_subgenre.dropna(subset=['sub_genre', 'subgenre_proportion_consumption'], how='all')
df_programmes = df_programme.dropna(subset=['programme_title', 'programme_proportion_consumption'], how='all')

In [10]:
#remove duplicated rows, if any

df_genre = df_genre.drop_duplicates(subset=None, keep='first', inplace=False)
df_subgenre = df_subgenre.drop_duplicates(subset=None, keep='first', inplace=False)
df_programme = df_programme.drop_duplicates(subset=None, keep='first', inplace=False)

In [11]:
#change necessary datatypes to float
df_genre.dtypes

viewer_id                       object
genre_id                        object
genre_proportion_consumption    object
dtype: object

In [12]:
df_genre['genre_proportion_consumption'] = pd.to_numeric(df_genre['genre_proportion_consumption'])
df_subgenre['subgenre_proportion_consumption'] = pd.to_numeric(df_subgenre['subgenre_proportion_consumption'])
df_programme['programme_proportion_consumption'] = pd.to_numeric(df_programme['programme_proportion_consumption'])

# Use This Function to Reduce the Datatypes to the Lowest Memory Version

In [13]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True
            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

In [14]:
props, NAlist = reduce_mem_usage(df_programme)
print("_________________")
print("")
print("Warning: the following columns have missing values filled with 'df['column_name'].min() -1': ")
print("_________________")
print("")
print(NAlist)

Memory usage of properties dataframe is : 60.879669189453125  MB
******************************
Column:  programme_proportion_consumption
dtype before:  float64
dtype after:  float32
******************************
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  53.269710540771484  MB
This is  87.5 % of the initial size
_________________

_________________

['programme_proportion_consumption']


In [15]:
#Pivot the genre/subgenre/programmes to show identicator values if they have watched a genre/subgenre/programme

df_genre2 = df_genre.pivot_table(index='viewer_id',columns='genre_id',values='genre_proportion_consumption').fillna(0)

In [16]:
df_subgenre2 = df_subgenre.pivot_table(index='viewer_id',columns='sub_genre',values='subgenre_proportion_consumption').fillna(0)

In [17]:
df_programme2 = df_programme.pivot_table(index='viewer_id',columns='programme_title',values='programme_proportion_consumption').fillna(0)

In [18]:
df_genre2.isnull().values.any()

False

In [19]:
df_subgenre2.isnull().values.any()

False

In [20]:
df_programme2.isnull().values.any()

False

In [21]:
df_demographics2 = df_demographics.loc[:,['viewer_id','age','gender','abc1_flag']]
df_demographics2.head()

,viewer_id,age,gender,abc1_flag
0,008e9dde-3724-4b38-afe8-ebe77a10936e,55.0,M,1
1,0199f9be-fc50-42d8-af74-8684a01ce8a3,34.0,M,1
2,01fdb9cd-355d-4cf1-a811-f78d53d0038e,92.0,M,0
3,026346f3-9f29-48c6-b71e-442f1b4fb971,52.0,F,1
4,027b4507-7a5a-42c2-b8ee-7a0893999298,38.0,M,1


In [22]:
df_demographics2 = df_demographics2.dropna(subset=['age', 'gender'], how='all')

In [23]:
# Convert to dummy variables
df_demographics_dummies = pd.concat([df_demographics2, pd.get_dummies(df_demographics2['gender']).astype('int32'),
                                     pd.get_dummies(df_demographics2['abc1_flag']).astype('int32')],axis=1)

In [24]:
df_demographics_dummies.head()

,viewer_id,age,gender,abc1_flag,F,M,X,0,1
0,008e9dde-3724-4b38-afe8-ebe77a10936e,55.0,M,1,0,1,0,0,1
1,0199f9be-fc50-42d8-af74-8684a01ce8a3,34.0,M,1,0,1,0,0,1
2,01fdb9cd-355d-4cf1-a811-f78d53d0038e,92.0,M,0,0,1,0,1,0
3,026346f3-9f29-48c6-b71e-442f1b4fb971,52.0,F,1,1,0,0,0,1
4,027b4507-7a5a-42c2-b8ee-7a0893999298,38.0,M,1,0,1,0,0,1


In [25]:
df_demographics_dummies = df_demographics_dummies.drop(['gender', 'abc1_flag'],axis=1)

In [27]:
# Join viewer data and demographics
from functools import reduce

df_demo_genre = reduce(lambda left,right: pd.merge(left,right,on='viewer_id'), 
                  [df_demographics_dummies, df_genre2, df_subgenre2, df_programme2])

df_demo_genre.head()

,viewer_id,age,F,M,X,0,1,ARTS,CHILDREN,COMEDY_x,...,Wycliffe,You VS. Chris & Kem,You're Only Young Twice,You've Been Framed!,Your Song,Yummy Mummies,Zoe Ball on...,easyJet: Inside The Cockpit,you've been t@gged,£10k Holiday Home
0,1ce3a23a-85a1-4c6d-8935-8f080d87a517,NaN,1,0,0,0,1,0.0,0.0000,0.0000,...,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5f04803b-a117-4414-b2cb-9c6c460b736f,NaN,1,0,0,0,1,0.0,0.0019,0.0000,...,0.0,0.0041,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6d4a5a13-b16c-4d87-a6bd-f5ea34aa7b38,44.0,1,0,0,1,0,0.0,0.0000,0.0016,...,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6daa982b-0a5a-43d5-aab2-f743cea9dc07,23.0,1,0,0,1,0,0.0,0.0000,0.0000,...,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,c2b74f68-fd35-422e-882d-38f95e4f3491,44.0,1,0,0,0,1,0.0,0.0000,0.0000,...,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
df_demo_genre.isnull().values.any()

True

In [29]:
df_demo_genre.isna().sum()

viewer_id                                                                       0
age                                                                         24954
F                                                                               0
M                                                                               0
X                                                                               0
0                                                                               0
1                                                                               0
ARTS                                                                            0
CHILDREN                                                                        0
COMEDY_x                                                                        0
DRAMA_x                                                                         0
ENT                                                                             0
FACTUAL         

In [31]:
df_demo_genre = df_demo_genre.dropna(subset=['age'], how='all')

In [33]:
props, NAlist = reduce_mem_usage(df_demo_genre)
print("_________________")
print("")
print("Warning: the following columns have missing values filled with 'df['column_name'].min() -1': ")
print("_________________")
print("")
print(NAlist)

Memory usage of properties dataframe is : 1341.3850402832031  MB
******************************
Column:  age
dtype before:  float64
dtype after:  uint16
******************************
******************************
Column:  F
dtype before:  int32
dtype after:  uint8
******************************
******************************
Column:  M
dtype before:  int32
dtype after:  uint8
******************************
******************************
Column:  X
dtype before:  int32
dtype after:  uint8
******************************
******************************
Column:  0
dtype before:  int32
dtype after:  uint8
******************************
******************************
Column:  1
dtype before:  int32
dtype after:  uint8
******************************
******************************
Column:  ARTS
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  CHILDREN
dtype before:  float64
dtype after:  float32
******************************


dtype after:  float32
******************************
******************************
Column:  HEALTH AND BEAUTY
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  HISTORY
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  HOBBIES AND INTERESTS
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  HOMES AND GARDENS
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  HORROR
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  HORSE RACING
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  LONG-RUNNING DRAMA
dtype before:  float64
dtype after:  float32
******************************
******************************

dtype after:  uint8
******************************
******************************
Column:  A Mother's Son
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  A Night for the Emergency Services
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  A Right Royal Quiz
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  A Touch of Frost
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  A Very Royal Wedding
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  A and E Live
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  A is for Acid
dtype before:  float32
dtype after:  uint8
******************************
*****

dtype after:  float32
******************************
******************************
Column:  Betch
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  Better Late Than Never
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  Big Star's Little Star
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  Bigheads
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  Billy Connolly & Me: A Celebration
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  Birds of a Feather
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  Birds of a Feather Christmas Special - House For the Rising Sons
dtype before:  float32
dtype af

dtype after:  float32
******************************
******************************
Column:  Cheerleaders Steel Rays
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  Chicago
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  Children Who Kill
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  Chocolat
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  Chopping Block
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  Chris & Kem Straight Outta Love Island
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  Chris & Olivia : Crackin' on
dtype before:  float32
dtype after:  float32
************************

...

dtype after:  uint8
******************************
******************************
Column:  Six Nations Championship
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  Six Nations: Round-up
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  So Sharp
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  Soccer Aid
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  Sooty
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  Source Code
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  Southern Charm
dtype before:  float32
dtype after:  float32
******************************
******************************
Column

dtype after:  float32
******************************
******************************
Column:  The Brighton Police
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  The British Soap Awards
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  The Car Chasers
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  The Chase
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  The Chase Blooper Show
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  The Chase: Celebrity Specials
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  The Classic Brit Awards
dtype before:  float32
dtype after:  float32
*******************

dtype after:  float32
******************************
******************************
Column:  The NHS Heroes Awards
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  The Nation's Favourite Elvis Song
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  The Nightly Show Best Bits
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  The Nightly Show with...
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  The Olivier Awards: The Biggest Night in British Theatre
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  The Only Way is Essex
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  The Only Way is Essexmas

dtype after:  float32
******************************
******************************
Column:  This Morning
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  This Morning - 30 Unforgettable Years
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  This Time Next Year
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  Through the Keyhole
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  Thunderbirds Are Go
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  Timewasters
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  Tina & Bobby
dtype before:  float32
dtype after:  float32
******************************

dtype after:  float32
******************************
******************************
Column:  Wishfart
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  Women Behind Bars with Trevor McDonald
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  World Boxing Super Series
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  World Boxing Super Series Support Shows
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  World Cup
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  World Cup Daily
dtype before:  float32
dtype after:  float32
******************************
******************************
Column:  World Cup Highlights
dtype before:  float32
dtype after:  float32
*****

In [36]:
del df_genre
del df_subgenre
del df_programme
del df_genre2
del df_subgenre2
del df_programme2
del df_viewers
del df_demographics
del df_demographics2
del df_demographics_dummies

NameError: name 'df_genre' is not defined

# PCA

In [38]:
scaler = StandardScaler()
df_demo_genre_scaled = scaler.fit_transform(df_demo_genre)

MemoryError: 

In [ ]:
pca = PCA()
df_demo_genre_reduced = pca.fit_transform(df_demo_genre_scaled)

In [ ]:
fig, ax = plt.subplots(figsize=(9,6))

ax.plot(pca.explained_variance_ratio_.cumsum())
ax.set_xlabel('n_components')
ax.set_ylabel('explained_variance_ratio');

In [ ]:
loadings_array = pca.components_.T * np.sqrt(pca.explained_variance_)
loadings = pd.DataFrame(loadings_array.T)
loadings.columns =  +  + 
loadings.index = loadings.index+1
loadings.head().transpose()

In [ ]:
fig, ax = plt.subplots(figsize=(10,20))
sns.heatmap(loadings.T, )
ax.set_xlabel('component');

In [ ]:
viewing_reduced_df = pd.DataFrame(viewing_reduced)
col_names = ["pc_" + str(x+1) for x in range(30)]
df_demo_genre_reduced_df.columns = col_names
print("Correlation of first PC with  is %s ")
print (df_demo_genre_reduced_df.pc_1.corr(pd.Series()))
print("Correlation of second PC with ")
print (df_demo_genre_reduced_df.pc_2.corr(pd.Series()))
print("Correlation of third PC with ")
print (df_demo_genre_reduced_df.pc_3.corr(pd.Series()))